# 손글씨/와인/유방암 데이터 분석

## 0. 준비

In [1]:
# 기본 라이브러리들
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 사이킷런의 모델들
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

# 사이킷런의 학습 평가 모듈들
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# 학습시킬 데이터들
from sklearn.datasets import load_digits
from sklearn.datasets import load_wine
from sklearn.datasets import load_breast_cancer

# 데이터 스플릿 모듈
from sklearn.model_selection import train_test_split

# LogisticRegression 모델을 실행할 때 지속적인 경고 창이 있어 우선 무시하도록 조치함
import warnings
warnings.filterwarnings('ignore')

## 1. 손글씨 분류

###  어떤 모델을 사용하는 것이 좋을까?

#### 데이터를 확인해보자

In [2]:
digits = load_digits()
print(dir(digits))

['DESCR', 'data', 'feature_names', 'frame', 'images', 'target', 'target_names']


In [3]:
dg_data = digits.data
print(dg_data.shape)

(1797, 64)


In [4]:
# shape을 보고 특성이 이렇게 많아? 라는 생각에 확인. 사실이었다..
len(digits.feature_names)

64

In [5]:
dg_data[0]

array([ 0.,  0.,  5., 13.,  9.,  1.,  0.,  0.,  0.,  0., 13., 15., 10.,
       15.,  5.,  0.,  0.,  3., 15.,  2.,  0., 11.,  8.,  0.,  0.,  4.,
       12.,  0.,  0.,  8.,  8.,  0.,  0.,  5.,  8.,  0.,  0.,  9.,  8.,
        0.,  0.,  4., 11.,  0.,  1., 12.,  7.,  0.,  0.,  2., 14.,  5.,
       10., 12.,  0.,  0.,  0.,  0.,  6., 13., 10.,  0.,  0.,  0.])

### digits

- 0에서 9까지의 손글씨 숫자 데이터

- 64가지 특성

음.. 일단 특성이 많으니까 나무 한 그루씩 그리기는 좋지 않을 것 같고..
SVM도 특성이 많은 것에 안 어울릴 것 같고.. 또 숫자 데이터는 생김새가 엄청 큰 차이가 없을 것 같으니 피처 간의 간격도 안 클 것 같고..

### !

![](./cheat.png)

모델에 대해서 아는 것이 없으니, 가이드를 따라가보자

카테고리가 있음 - 라벨링 된 데이터 - 10만개 보다 작음 = SGD Classifier

In [6]:
from sklearn.linear_model import SGDClassifier
sgd_model = SGDClassifier()

print(sgd_model._estimator_type)

classifier


In [7]:
dg_label = digits.target

In [8]:
X_train, X_test, y_train, y_test = train_test_split(dg_data, 
                                                    dg_label,
                                                    random_state=123,
                                                    test_size=0.2)

In [9]:
sgd_model = SGDClassifier()
sgd_model.fit(X_train, y_train)
y_pred = sgd_model.predict(X_test)

### 평가 지표 선정

- 라벨과 1:1로 대응하는 손글씨 이미지가 무엇인지 예측하는 것이 목표
- 전체 데이터 대비 정답으로 예측한 비율, 즉 정확도가 적합해 보임

In [10]:
#모델 간 비교를 위해 정확도는 리스트에 담아주자.
sgd_acc = [accuracy_score(y_test, y_pred)]
digits_acc = []
digits_acc += [sgd_acc]

print("SDG_accuracy:", sgd_acc)
print(classification_report(y_test, y_pred))

SDG_accuracy: [0.9416666666666667]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        39
           1       0.84      0.94      0.89        34
           2       0.86      1.00      0.92        36
           3       0.94      0.97      0.96        33
           4       0.95      0.98      0.96        42
           5       1.00      0.86      0.93        37
           6       1.00      0.95      0.98        43
           7       0.91      1.00      0.95        31
           8       1.00      0.73      0.84        37
           9       0.93      1.00      0.97        28

    accuracy                           0.94       360
   macro avg       0.94      0.94      0.94       360
weighted avg       0.95      0.94      0.94       360



전반적인 점수가 높게 나왔다. 역시 치트시트의 힘인가?

### 나머지 모델도 사용해보자

#### Decision Tree

In [11]:
tree_model = DecisionTreeClassifier()
tree_model.fit(X_train, y_train)
y_pred = tree_model.predict(X_test)

In [12]:
tree_acc = [accuracy_score(y_test, y_pred)]
digits_acc += [tree_acc]

print("tree_accuracy:", tree_acc)
print(classification_report(y_test, y_pred))

tree_accuracy: [0.8277777777777777]
              precision    recall  f1-score   support

           0       0.92      0.92      0.92        39
           1       0.69      0.85      0.76        34
           2       0.82      0.78      0.80        36
           3       0.82      0.82      0.82        33
           4       0.85      0.79      0.81        42
           5       0.93      0.76      0.84        37
           6       0.93      0.91      0.92        43
           7       0.82      0.90      0.86        31
           8       0.81      0.70      0.75        37
           9       0.69      0.86      0.76        28

    accuracy                           0.83       360
   macro avg       0.83      0.83      0.82       360
weighted avg       0.84      0.83      0.83       360



In [13]:
forest_model = RandomForestClassifier()
forest_model.fit(X_train, y_train)
y_pred = forest_model.predict(X_test)

In [14]:
forest_acc  = [accuracy_score(y_test, y_pred)]
digits_acc += [forest_acc]

print("forest_accuracy:", forest_acc)
print(classification_report(y_test, y_pred))

forest_accuracy: [0.9777777777777777]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        39
           1       0.89      1.00      0.94        34
           2       1.00      1.00      1.00        36
           3       1.00      1.00      1.00        33
           4       0.98      0.98      0.98        42
           5       0.97      0.95      0.96        37
           6       1.00      0.95      0.98        43
           7       0.97      1.00      0.98        31
           8       1.00      0.92      0.96        37
           9       0.97      1.00      0.98        28

    accuracy                           0.98       360
   macro avg       0.98      0.98      0.98       360
weighted avg       0.98      0.98      0.98       360



In [15]:
svm_model = svm.SVC()
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)

In [16]:
svm_acc = [accuracy_score(y_test, y_pred)]
digits_acc += [svm_acc]

print("svm_accuracy:", svm_acc)
print(classification_report(y_test, y_pred))

svm_accuracy: [0.9888888888888889]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        39
           1       0.92      1.00      0.96        34
           2       1.00      1.00      1.00        36
           3       1.00      1.00      1.00        33
           4       1.00      1.00      1.00        42
           5       1.00      0.97      0.99        37
           6       1.00      1.00      1.00        43
           7       1.00      1.00      1.00        31
           8       1.00      0.92      0.96        37
           9       0.97      1.00      0.98        28

    accuracy                           0.99       360
   macro avg       0.99      0.99      0.99       360
weighted avg       0.99      0.99      0.99       360



In [17]:
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)
y_pred = logistic_model.predict(X_test)

In [18]:
logistic_acc = [accuracy_score(y_test, y_pred)]
digits_acc += [logistic_acc]

print("logistic_accuracy:", svm_acc)
print(classification_report(y_test, y_pred))

logistic_accuracy: [0.9888888888888889]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        39
           1       0.85      1.00      0.92        34
           2       0.97      1.00      0.99        36
           3       1.00      1.00      1.00        33
           4       0.98      0.98      0.98        42
           5       0.97      0.89      0.93        37
           6       1.00      0.95      0.98        43
           7       0.97      1.00      0.98        31
           8       1.00      0.89      0.94        37
           9       0.97      1.00      0.98        28

    accuracy                           0.97       360
   macro avg       0.97      0.97      0.97       360
weighted avg       0.97      0.97      0.97       360



In [19]:
digits_acc

[[0.9416666666666667],
 [0.8277777777777777],
 [0.9777777777777777],
 [0.9888888888888889],
 [0.9694444444444444]]

### ????
근소한 차이지만 SVM모델의 정확도가 가장 높게 나왔다.

## 2. 와인 분류

In [20]:
wine = load_wine()
print(dir(wine))

['DESCR', 'data', 'feature_names', 'frame', 'target', 'target_names']


와인 데이터는 총 178개, 각각 13개의 특성.

In [21]:
wine_data = wine.data
wine_data.shape

(178, 13)

In [22]:
wine_label = wine.target
wine_label.shape

(178,)

### 앞서의 교훈은 우선 모든 모델을 사용해보자는 것이었다.

### 평가 지표

- digits와 마찬가지의 이유로 정확도로 판단하는 것이 합당해 보인다

In [23]:
X_train, X_test, y_train, y_test = train_test_split(wine_data, 
                                                    wine_label,
                                                    random_state=123,
                                                    test_size=0.2)

### 배운 것을 써먹어보자

In [24]:
model_list = [DecisionTreeClassifier, RandomForestClassifier, svm.SVC, SGDClassifier, LogisticRegression]

model_list
    

[sklearn.tree._classes.DecisionTreeClassifier,
 sklearn.ensemble._forest.RandomForestClassifier,
 sklearn.svm._classes.SVC,
 sklearn.linear_model._stochastic_gradient.SGDClassifier,
 sklearn.linear_model._logistic.LogisticRegression]

In [25]:
wine_acc=[]
for i in model_list:
    model = i()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print(accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    wine_acc.append(accuracy_score(y_test, y_pred))

0.9444444444444444
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       0.85      1.00      0.92        11
           2       1.00      0.88      0.94        17

    accuracy                           0.94        36
   macro avg       0.95      0.96      0.95        36
weighted avg       0.95      0.94      0.95        36

1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        11
           2       1.00      1.00      1.00        17

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36

0.5
              precision    recall  f1-score   support

           0       0.80      1.00      0.89         8
           1       0.38      0.91      0.54        11
           2       0.00      0.00      0.00     

In [26]:
for acc in wine_acc:
    print(acc)

0.9444444444444444
1.0
0.5
0.5277777777777778
1.0


### 결과

- 랜덤포레스트와 선형회귀가 우승.
- 하지만 선형회귀는 계속 경고가 뜬다.
- 이정도 정확도라면 가장 가벼운 의사결정나무를 사용해도 좋을 것 같다.

## 3. 유방암 진단

In [27]:
#데이터 불러오기
breast_cancer = load_breast_cancer()
#이번엔 배운대로 키값을 호출해보자
breast_cancer.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [28]:
breast_cancer_data = breast_cancer.data
print(breast_cancer_data.shape)
breast_cancer_label = breast_cancer.target
print(breast_cancer_label.shape)

(569, 30)
(569,)


569개의 데이터, 30개의 특성

### 평가 지표

- 암 진단의 경우, 악성을 양성으로 잘못 예측한 경우의 피해가 막심하다.
- 그렇다면 어떤 평가 지표가 타당할까?

![](./E-2-3.jpg)

- True가 악성(암 입니다..)임에 유의해야 한다.
- 결과적으로 양성을 음성으로 판단한 경우가 가장 치명적이다

> Recall(Sensitivity)은 분모에 있는 FNFN이 낮을수록 커집니다. Recall이 높아지려면 False Negative, 즉 양성인데 음성으로 판단하는 경우가 적어야 합니다.

### 학습

- 이전의 방식에서 정확도 스코어만 리콜로 바꿀 수 있으면 좋을 것 같다.
- 찾아보니 사이킷런 패키지님은 리콜스코어도 제공한다.
> recall_score(y_true, y_pred)


In [29]:
X_train, X_test, y_train, y_test = train_test_split(breast_cancer_data, 
                                                    breast_cancer_label,
                                                    random_state=123,
                                                    test_size=0.2)

In [30]:
model_list

[sklearn.tree._classes.DecisionTreeClassifier,
 sklearn.ensemble._forest.RandomForestClassifier,
 sklearn.svm._classes.SVC,
 sklearn.linear_model._stochastic_gradient.SGDClassifier,
 sklearn.linear_model._logistic.LogisticRegression]

In [31]:
from sklearn.metrics import recall_score

In [32]:
cancer_recall=[]
for i in model_list:
    model = i()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print(recall_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    cancer_recall.append(recall_score(y_test, y_pred))

0.9863013698630136
              precision    recall  f1-score   support

           0       0.97      0.95      0.96        41
           1       0.97      0.99      0.98        73

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114

1.0
              precision    recall  f1-score   support

           0       1.00      0.98      0.99        41
           1       0.99      1.00      0.99        73

    accuracy                           0.99       114
   macro avg       0.99      0.99      0.99       114
weighted avg       0.99      0.99      0.99       114

1.0
              precision    recall  f1-score   support

           0       1.00      0.80      0.89        41
           1       0.90      1.00      0.95        73

    accuracy                           0.93       114
   macro avg       0.95      0.90      0.92       114
weighted avg       0.94      0.93      0.93    

In [33]:
for recall in cancer_recall:
    print(recall)

0.9863013698630136
1.0
1.0
0.9178082191780822
1.0


### ????

- 결과를 보니 뭔가 이상하다.
- 정확한 원인을 판단할 수 없고, 시간이 부족하니 우선 분류 성능 평가 표의 '0'인덱스 리콜을 살펴보자

In [34]:
for i in model_list:
    model = i()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.95      0.96        41
           1       0.97      0.99      0.98        73

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        41
           1       0.99      1.00      0.99        73

    accuracy                           0.99       114
   macro avg       0.99      0.99      0.99       114
weighted avg       0.99      0.99      0.99       114

              precision    recall  f1-score   support

           0       1.00      0.80      0.89        41
           1       0.90      1.00      0.95        73

    accuracy                           0.93       114
   macro avg       0.95      0.90      0.92       114
weighted avg       0.94      0.93      0.93       114

              preci

In [35]:
classification_report

<function sklearn.metrics._classification.classification_report(y_true, y_pred, *, labels=None, target_names=None, sample_weight=None, digits=2, output_dict=False, zero_division='warn')>

### 결과

- 랜덤포레스트가 가장 높은 리콜 점수를 보인다.

## 4. 마무리

- Exp1에서 너무 오랜 시간을 지체하여 마감에 쫓겨 아쉽다.
- recall_score를 바로 확인하는 방법을 알아보아야겠다.